<a href="https://colab.research.google.com/github/KNGLJordan/CDMO-project/blob/main/src/SAT/sat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install python3-z3

[sudo] password for lucazini03: 


In [4]:
from z3 import *
from itertools import combinations

In [2]:
#the man the myth the legend: angelo quarta

#NAIVE PAIRWISE
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars, name = ""):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars, name = ""):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars, name))

def at_least_k_np(bool_vars, k):
    return at_most_k_np([Not(var) for var in bool_vars], len(bool_vars)-k)

def at_most_k_np(bool_vars, k):
    return And([Or([Not(x) for x in X]) for X in combinations(bool_vars, k + 1)])

def exactly_k_np(bool_vars, k):
    return And(at_most_k_np(bool_vars, k), at_least_k_np(bool_vars, k))


#SEQUENTIAL
def at_least_one_seq(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_seq(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one_seq(bool_vars, name):
    return And(at_least_one_seq(bool_vars), at_most_one_seq(bool_vars, name))

def at_least_k_seq(bool_vars, k, name):
    return at_most_k_seq([Not(var) for var in bool_vars], len(bool_vars)-k, name)

def at_most_k_seq(bool_vars, k, name):
    constraints = []
    n = len(bool_vars)
    s = [[Bool(f"s_{name}_{i}_{j}") for j in range(k)] for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0][0]))
    constraints += [Not(s[0][j]) for j in range(1, k)]
    for i in range(1, n-1):
        constraints.append(Or(Not(bool_vars[i]), s[i][0]))
        constraints.append(Or(Not(s[i-1][0]), s[i][0]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][k-1])))
        for j in range(1, k):
            constraints.append(Or(Not(bool_vars[i]), Not(s[i-1][j-1]), s[i][j]))
            constraints.append(Or(Not(s[i-1][j]), s[i][j]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2][k-1])))
    return And(constraints)

def exactly_k_seq(bool_vars, k, name):
    return And(at_most_k_seq(bool_vars, k, name), at_least_k_seq(bool_vars, k, name))

#HEULE
def at_least_one_he(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_he(bool_vars, name):
    if len(bool_vars) <= 4:
        return And(at_most_one_np(bool_vars))
    y = Bool(f"y_{name}")
    return And(And(at_most_one_np(bool_vars[:3] + [y])), And(at_most_one_he(bool_vars[3:] + [Not(y)], name+"_")))

def exactly_one_he(bool_vars, name):
    return And(at_most_one_he(bool_vars, name), at_least_one_he(bool_vars))

#BINARY
def toBinary(num, length = None):
    num_bin = bin(num).split("b")[-1]
    if length:
        return "0"*(length - len(num_bin)) + num_bin
    return num_bin
    
def at_least_one_bw(bool_vars):
    return at_least_one_np(bool_vars)

def at_most_one_bw(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    m = math.ceil(math.log2(n))
    r = [Bool(f"r_{name}_{i}") for i in range(m)]
    binaries = [toBinary(i, m) for i in range(n)]
    for i in range(n):
        for j in range(m):
            phi = Not(r[j])
            if binaries[i][j] == "1":
                phi = r[j]
            constraints.append(Or(Not(bool_vars[i]), phi))        
    return And(constraints)

def exactly_one_bw(bool_vars, name):
    return And(at_least_one_bw(bool_vars), at_most_one_bw(bool_vars, name)) 

In [4]:
from itertools import combinations
from z3 import *
import time

import numpy as np

# Get number of teams from user
n = int(input("Enter even number of teams: "))

# Start timer
start_time = time.time()

# === Build schedule ===
if n % 2 != 0:
    raise ValueError("Number of teams must be even.")

weeks = n - 1
periods = n // 2
teams = list(range(n))


home = np.empty((n, weeks, periods), dtype=object)
away = np.empty((n, weeks, periods), dtype=object)
for t in teams:
    for w in range(weeks):
        for p in range(periods):
            home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
            away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

s = Solver()

# #seems to be slowing down execution actually
# for p in range(periods):
#     i, j = 2*p, 2*p + 1  # pairing (0 vs 1), (2 vs 3), etc.
#     s.add(home[i, 0, p])
#     s.add(away[j, 0, p])

# Make team 0 always home in period 0 of week 0
# s.add(home[0, 0, 0])

for w in range(weeks - 1):
    for p in range(periods - 1):
        s.add(Implies(home[0, w, p], home[0, w + 1, (p + 1) % periods]))



# 0. Each slot has exactly one home and one away
for w in range(weeks):
    for p in range(periods):
        s.add(exactly_one_he([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
        s.add(exactly_one_he([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

# 1. Every team plays exactly once per week
for t in teams:
    for w in range(weeks):
        vars_in_week = [home[t, w, p] for p in range(periods)] + \
                       [away[t, w, p] for p in range(periods)]
        s.add(exactly_one_he(vars_in_week, name=f"team_plays_{t}_{w}"))

# 2. Every pair meets exactly once
for i, j in combinations(teams, 2):
    match_slots = []
    for w in range(weeks):
        for p in range(periods):
            match_slots.append(
                Or(And(home[i, w, p], away[j, w, p]),
                   And(home[j, w, p], away[i, w, p]))
            )
    s.add(exactly_one_he(match_slots, name=f"pair_meets_{i}_{j}"))

# # 3. At most 2 games per team per period
# for t in teams:
#     for p in range(periods):
#         games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
#         for combo in combinations(games, 3):
#             s.add(Not(And(combo)))

for t in teams:
    for p in range(periods):
        games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
        s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))


# Solve
if s.check() != sat:
    sched = None
else:
    m = s.model()

    # Extract schedule
    sched = [[None]*weeks for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if m.evaluate(home[i, w, p]):
                    for j in teams:
                        if m.evaluate(away[j, w, p]):
                            sched[p][w] = (i+1, j+1)

# End timer
end_time = time.time()

# Print schedule
if sched is None:
    print("No solution.")
else:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1): print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()

print(f"\nExecution time: {end_time - start_time:.2f} seconds")

          Week 1    Week 2    Week 3    Week 4    Week 5    Week 6    Week 7    Week 8    Week 9   Week 10   Week 11  
Period 1:  12 v 11    6 v 11    9 v 4     8 v 10    4 v 2     10 v 3    3 v 5     7 v 2     9 v 5     12 v 7    1 v 6   
Period 2:   8 v 1     4 v 3     6 v 3     11 v 9   10 v 11    6 v 8     10 v 1    9 v 12    12 v 2    5 v 2     7 v 5   
Period 3:   3 v 2     1 v 12    5 v 11    3 v 7     9 v 8     4 v 7     6 v 12    4 v 1     8 v 11    9 v 10    2 v 10  
Period 4:   4 v 5     7 v 9     1 v 2     6 v 2     5 v 6     9 v 1     7 v 11    11 v 3    4 v 10    8 v 3     8 v 12  
Period 5:   10 v 7    8 v 2     7 v 8     1 v 5     3 v 12    5 v 12    9 v 2     6 v 10    1 v 3     6 v 4     4 v 11  
Period 6:   6 v 9     5 v 10   12 v 10    4 v 12    1 v 7     2 v 11    8 v 4     8 v 5     6 v 7     1 v 11    3 v 9   

Execution time: 14.75 seconds


In [6]:
from itertools import combinations
from z3 import *
import time

import numpy as np

# Get number of teams from user
n = int(input("Enter even number of teams: "))

# Start timer
start_time = time.time()

# === Build schedule ===
if n % 2 != 0:
    raise ValueError("Number of teams must be even.")

weeks = n - 1
periods = n // 2
teams = list(range(n))


home = np.empty((n, weeks, periods), dtype=object)
away = np.empty((n, weeks, periods), dtype=object)
for t in teams:
    for w in range(weeks):
        for p in range(periods):
            home[t, w, p] = Bool(f"H_{t}_{w}_{p}")
            away[t, w, p] = Bool(f"A_{t}_{w}_{p}")

s = Solver()

# #seems to be slowing down execution actually
# for p in range(periods):
#     i, j = 2*p, 2*p + 1  # pairing (0 vs 1), (2 vs 3), etc.
#     s.add(home[i, 0, p])
#     s.add(away[j, 0, p])

# Make team 0 always home in period 0 of week 0
# s.add(home[0, 0, 0])

# for w in range(weeks - 1):
#     for p in range(periods - 1):
#         s.add(Implies(home[0, w, p], home[0, w + 1, (p + 1) % periods]))



# 0. Each slot has exactly one home and one away
for w in range(weeks):
    for p in range(periods):
        s.add(exactly_one_he([home[t, w, p] for t in teams], name=f"slot_home_{w}_{p}"))
        s.add(exactly_one_he([away[t, w, p] for t in teams], name=f"slot_away_{w}_{p}"))

# 1. Every team plays exactly once per week
for t in teams:
    for w in range(weeks):
        vars_in_week = [home[t, w, p] for p in range(periods)] + \
                       [away[t, w, p] for p in range(periods)]
        s.add(exactly_one_he(vars_in_week, name=f"team_plays_{t}_{w}"))

# 2. Every pair meets exactly once
for i, j in combinations(teams, 2):
    match_slots = []
    for w in range(weeks):
        for p in range(periods):
            match_slots.append(
                Or(And(home[i, w, p], away[j, w, p]),
                   And(home[j, w, p], away[i, w, p]))
            )
    s.add(exactly_one_he(match_slots, name=f"pair_meets_{i}_{j}"))

# # 3. At most 2 games per team per period
# for t in teams:
#     for p in range(periods):
#         games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
#         for combo in combinations(games, 3):
#             s.add(Not(And(combo)))

for t in teams:
    for p in range(periods):
        games = [home[t, w, p] for w in range(weeks)] + [away[t, w, p] for w in range(weeks)]
        s.add(at_most_k_seq(games, 2, name=f"team_{t}_period_{p}"))


# Solve
if s.check() != sat:
    sched = None
else:
    m = s.model()

    # Extract schedule
    sched = [[None]*weeks for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for i in teams:
                if m.evaluate(home[i, w, p]):
                    for j in teams:
                        if m.evaluate(away[j, w, p]):
                            sched[p][w] = (i+1, j+1)

# End timer
end_time = time.time()

# Print schedule
if sched is None:
    print("No solution.")
else:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1): print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()

print(f"\nExecution time: {end_time - start_time:.2f} seconds")

          Week 1    Week 2    Week 3    Week 4    Week 5    Week 6    Week 7    Week 8    Week 9   Week 10   Week 11   Week 12   Week 13  
Period 1:  14 v 10   10 v 12   12 v 11    14 v 9    1 v 13    5 v 1     7 v 9     2 v 6     2 v 7     4 v 11    3 v 4     3 v 5     8 v 13  
Period 2:   12 v 9    4 v 14    6 v 10    7 v 13    8 v 12    2 v 3     8 v 5     1 v 7     3 v 6     1 v 9     11 v 5   11 v 14    10 v 4  
Period 3:   3 v 7     2 v 9     13 v 9    6 v 4     5 v 2    13 v 10    11 v 3    14 v 8    5 v 12    6 v 14    7 v 12    8 v 10    1 v 11  
Period 4:   2 v 1     3 v 8     5 v 14    5 v 10    6 v 11    11 v 9    13 v 4    12 v 4    10 v 9    8 v 2     6 v 13    1 v 12    14 v 7  
Period 5:   4 v 5     13 v 5    8 v 7     8 v 1     3 v 14    6 v 7     2 v 10   11 v 10    4 v 1    13 v 12    14 v 2    6 v 9     3 v 9   
Period 6:  13 v 11    6 v 1     1 v 3     11 v 2    7 v 10    4 v 8     6 v 12    5 v 9    13 v 14    3 v 10    8 v 9     7 v 4     2 v 12  
Period 7:   6 v

# SIMILAR TO NURSE SCHEDULING OF ANGELO QUARTA (sucks)

In [19]:
from itertools import combinations
from z3 import *
import time

# Get number of teams from user
n = int(input("Enter even number of teams: "))
if n % 2 != 0:
    raise ValueError("Number of teams must be even.")

# Start timer
start_time = time.time()

weeks = n - 1
periods = n // 2
teams = list(range(n))

# SAT solver
s = Solver()

# Create variables using 4D structure similar to nurse scheduling
games = [[[[Bool(f"x_{t1}_{t2}_{w}_{p}") for p in range(periods)] for w in range(weeks)] for t2 in teams] for t1 in teams]

# 1. Each pair plays once (t1 vs t2 or t2 vs t1 over any week and period)
for t1, t2 in combinations(teams, 2):
    match_occurrences = []
    for w in range(weeks):
        for p in range(periods):
            match_occurrences.append(games[t1][t2][w][p])
            match_occurrences.append(games[t2][t1][w][p])
    s.add(Or(match_occurrences))

# 2. One match per team per week
for t in teams:
    for w in range(weeks):
        matches = []
        for t2 in teams:
            if t != t2:
                for p in range(periods):
                    matches.append(games[t][t2][w][p])
                    matches.append(games[t2][t][w][p])
        s.add(at_most_k_seq(matches, 1, name=f"team_{t}_week_{w}"))

# 3. Each period has exactly one match per week (i.e., one home team per period/week)
for w in range(weeks):
    for p in range(periods):
        games_in_period = []
        for t1 in teams:
            for t2 in teams:
                if t1 != t2:
                    games_in_period.append(games[t1][t2][w][p])
        s.add(at_most_k_seq(games_in_period, 1, name=f"period_{w}_{p}"))
        s.add(Or(games_in_period))  # at least one, so exactly one

# 4. A team appears in each period at most twice over all weeks
for t in teams:
    for p in range(periods):
        appears = []
        for w in range(weeks):
            for opp in teams:
                if t != opp:
                    appears.append(games[t][opp][w][p])
                    appears.append(games[opp][t][w][p])
        s.add(at_most_k_seq(appears, 2, name=f"team_{t}_period_{p}"))

for p in range(periods):
    i, j = 2*p, 2*p+1
    s.add(home[i, 0, p])
    s.add(away[j, 0, p])

# Solve
if s.check() != sat:
    print("No solution.")
    sched = None
else:
    m = s.model()
    sched = [[None for _ in range(weeks)] for _ in range(periods)]
    for w in range(weeks):
        for p in range(periods):
            for t1 in teams:
                for t2 in teams:
                    if t1 != t2 and m.evaluate(games[t1][t2][w][p], model_completion=True):
                        sched[p][w] = (t1 + 1, t2 + 1)

# Print schedule
if sched is None:
    print("No solution.")
else:
    weeks = len(sched[0])
    print("        ", end="")
    for w in range(1, weeks+1): print(f"Week {w}".center(10), end="")
    print()
    for p, row in enumerate(sched, 1):
        print(f"Period {p}:", end=" ")
        for game in row:
            print(f"{game[0]} v {game[1]}".center(10), end="")
        print()

# End timer
end_time = time.time()
print(f"\nExecution time: {end_time - start_time:.2f} seconds")

No solution.
No solution.

Execution time: 53.48 seconds
